### ETL del archivo en crudo `movies_dataset.parquet` con las columnas de datos simples. El nuevo dataset se va a llamar `movies`

In [10]:
import pandas as pd
import ast
import os
import gc
import json

In [11]:
url = "https://github.com/FranciscoHugoLezik/Movies_data/blob/main/movies_dataset.parquet?raw=true"

movies_dataset = (
    pd.read_parquet(
        url, 
        engine="fastparquet"
        )
    )

In [12]:
movies_dataset.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [13]:
recorte = [
    "video", 
    "imdb_id", 
    "adult", 
    "original_title", 
    "poster_path", 
    "homepage", 
    "belongs_to_collection", 
    "production_companies", 
    "production_countries", 
    "spoken_languages"
    ]

movies_recortado = (
    movies_dataset
    .drop(columns=recorte)
    .copy()
    )

In [14]:
del movies_dataset
gc.collect()

852

In [15]:
for columna in movies_recortado.columns:
    print(columna)

budget
genres
id
original_language
overview
popularity
release_date
revenue
runtime
status
tagline
title
vote_average
vote_count


In [16]:
movies_recortado.rename(
    columns={'id': 'movie_id'}, 
    inplace=True
    )

In [17]:
('movie_id' in movies_recortado.columns 
 and 'id' not in movies_recortado.columns)

True

In [18]:
nested_genres = (
    movies_recortado
    [['genres', 
      'movie_id']]
    .copy()
    )

In [19]:
movies_recortado.drop(
  columns='genres', 
  inplace=True)

In [20]:
nested_genres.iloc[0]

genres      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
movie_id                                                  862
Name: 0, dtype: object

In [21]:
nested_genres.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [22]:
nested_genres.rename(
    columns={'id': 'movie_id'}, 
    inplace=True
    )

In [23]:
nested_genres.columns

Index(['genres', 'movie_id'], dtype='object')

In [24]:
nested_genres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    45466 non-null  object
 1   movie_id  45466 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [25]:
generos_por_pelicula = [
    {**generos, 'movie_id': row['movie_id']}
    for _, row in nested_genres.iterrows()
    for generos in ast.literal_eval(row['genres'])
]

generos = pd.DataFrame(generos_por_pelicula)

In [26]:
del nested_genres
del generos_por_pelicula
gc.collect()

1111

In [27]:
generos.head()

,id,name,movie_id
0,16,Animation,862
1,35,Comedy,862
2,10751,Family,862
3,12,Adventure,8844
4,14,Fantasy,8844


In [28]:
generos.drop(
    columns='id', 
    inplace=True
    )

In [29]:
generos.columns

Index(['name', 'movie_id'], dtype='object')

In [30]:
generos.rename(
    columns={'name': 'genre'}, 
    inplace=True
    )

In [31]:
generos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91106 entries, 0 to 91105
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre     91106 non-null  object
 1   movie_id  91106 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [32]:
generos_por_pelicula = (
    generos
    .groupby('movie_id')
    .agg(lambda genre: tuple(genre))
    .reset_index()
    )

In [33]:
del generos
gc.collect()

0

In [34]:
generos_por_pelicula.head()

,movie_id,genre
0,100,"(Comedy, Crime)"
1,10000,"(Comedy, Drama)"
2,10001,"(Comedy, Science Fiction)"
3,100010,"(Drama, War)"
4,100017,"(Drama,)"


In [35]:
generos_por_pelicula['genre'] = (
    generos_por_pelicula['genre']
    .apply(lambda x: json.dumps(x)))

In [36]:
generos_por_pelicula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42994 entries, 0 to 42993
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  42994 non-null  object
 1   genre     42994 non-null  object
dtypes: object(2)
memory usage: 671.9+ KB


In [37]:
movies = pd.merge(
    movies_recortado, 
    generos_por_pelicula, 
    how='inner', 
    on='movie_id'
    )

In [38]:
del movies_recortado
del generos_por_pelicula
gc.collect()

0

In [39]:
movies.iloc[0]

budget                                                        30000000
movie_id                                                           862
original_language                                                   en
overview             Led by Woody, Andy's toys live happily in his ...
popularity                                                   21.946943
release_date                                                1995-10-30
revenue                                                      373554033
runtime                                                           81.0
status                                                        Released
tagline                                                           None
title                                                        Toy Story
vote_average                                                       7.7
vote_count                                                        5415
genre                                ["Animation", "Comedy", "Family"]
Name: 

In [40]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43024 entries, 0 to 43023
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   budget             43024 non-null  object
 1   movie_id           43024 non-null  object
 2   original_language  43014 non-null  object
 3   overview           42327 non-null  object
 4   popularity         43019 non-null  object
 5   release_date       42995 non-null  object
 6   revenue            43018 non-null  object
 7   runtime            42839 non-null  object
 8   status             42967 non-null  object
 9   tagline            20146 non-null  object
 10  title              43018 non-null  object
 11  vote_average       43018 non-null  object
 12  vote_count         43018 non-null  object
 13  genre              43024 non-null  object
dtypes: object(14)
memory usage: 4.9+ MB


In [41]:
movies.dropna(
    subset='release_date', 
    inplace=True
    )

In [42]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42995 entries, 0 to 43023
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   budget             42995 non-null  object
 1   movie_id           42995 non-null  object
 2   original_language  42985 non-null  object
 3   overview           42301 non-null  object
 4   popularity         42993 non-null  object
 5   release_date       42995 non-null  object
 6   revenue            42992 non-null  object
 7   runtime            42816 non-null  object
 8   status             42942 non-null  object
 9   tagline            20138 non-null  object
 10  title              42992 non-null  object
 11  vote_average       42992 non-null  object
 12  vote_count         42992 non-null  object
 13  genre              42995 non-null  object
dtypes: object(14)
memory usage: 4.9+ MB


In [43]:
movies.dropna(
    subset='title', 
    inplace=True
    )

In [44]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42992 entries, 0 to 43023
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   budget             42992 non-null  object
 1   movie_id           42992 non-null  object
 2   original_language  42982 non-null  object
 3   overview           42298 non-null  object
 4   popularity         42992 non-null  object
 5   release_date       42992 non-null  object
 6   revenue            42992 non-null  object
 7   runtime            42816 non-null  object
 8   status             42942 non-null  object
 9   tagline            20138 non-null  object
 10  title              42992 non-null  object
 11  vote_average       42992 non-null  object
 12  vote_count         42992 non-null  object
 13  genre              42992 non-null  object
dtypes: object(14)
memory usage: 4.9+ MB


In [45]:
movies['budget'] = (
    movies['budget']
    .astype('int64')
    )

movies['popularity'] = (
    movies['popularity']
    .astype('float64')
    )

movies['revenue'] = (
    movies['revenue']
    .astype('int64')
    )

movies['runtime'] = (
    movies['runtime']
    .astype('float64')
    )

movies['vote_average'] = (
    movies['vote_average']
    .astype('float64')
    )

movies['vote_count'] = (
    movies['vote_count']
    .astype('int64')
    )

In [46]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42992 entries, 0 to 43023
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             42992 non-null  int64  
 1   movie_id           42992 non-null  object 
 2   original_language  42982 non-null  object 
 3   overview           42298 non-null  object 
 4   popularity         42992 non-null  float64
 5   release_date       42992 non-null  object 
 6   revenue            42992 non-null  int64  
 7   runtime            42816 non-null  float64
 8   status             42942 non-null  object 
 9   tagline            20138 non-null  object 
 10  title              42992 non-null  object 
 11  vote_average       42992 non-null  float64
 12  vote_count         42992 non-null  int64  
 13  genre              42992 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 4.9+ MB


In [47]:
movies.reset_index(
    drop=True, 
    inplace=True
    )

In [48]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42992 entries, 0 to 42991
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             42992 non-null  int64  
 1   movie_id           42992 non-null  object 
 2   original_language  42982 non-null  object 
 3   overview           42298 non-null  object 
 4   popularity         42992 non-null  float64
 5   release_date       42992 non-null  object 
 6   revenue            42992 non-null  int64  
 7   runtime            42816 non-null  float64
 8   status             42942 non-null  object 
 9   tagline            20138 non-null  object 
 10  title              42992 non-null  object 
 11  vote_average       42992 non-null  float64
 12  vote_count         42992 non-null  int64  
 13  genre              42992 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 4.6+ MB


In [49]:
movies['revenue'] = (
    movies['revenue']
    .fillna(0)
    )

movies['budget'] = (
    movies['budget']
    .fillna(0)
    )

In [50]:
movies['release_year'] = (
    movies['release_date']
    .astype('datetime64[ns]')
    .dt
    .year
    .astype(str)
    )

In [51]:
movies.iloc[0]['release_date']

'1995-10-30'

In [52]:
movies.iloc[0]['release_year']

'1995'

In [53]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42992 entries, 0 to 42991
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             42992 non-null  int64  
 1   movie_id           42992 non-null  object 
 2   original_language  42982 non-null  object 
 3   overview           42298 non-null  object 
 4   popularity         42992 non-null  float64
 5   release_date       42992 non-null  object 
 6   revenue            42992 non-null  int64  
 7   runtime            42816 non-null  float64
 8   status             42942 non-null  object 
 9   tagline            20138 non-null  object 
 10  title              42992 non-null  object 
 11  vote_average       42992 non-null  float64
 12  vote_count         42992 non-null  int64  
 13  genre              42992 non-null  object 
 14  release_year       42992 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 4.9+ MB


In [54]:
return_movie = (
    lambda row: 
        row['revenue'] / row['budget'] 
    if row['revenue'] > 0 
    and row['budget'] > 0 
    else 0
    )

movies['return'] = (
    movies
    .apply(return_movie, 
           axis=1)
    .round(2)
    )

In [55]:
con_retorno = (movies['return'] != 0).sum()
con_retorno

5308

In [56]:
sin_retorno = (movies['return'] == 0).sum()
sin_retorno

37684

In [57]:
con_retorno + sin_retorno

42992

In [58]:
current_dir = os.getcwd()

current_dir

'c:\\Users\\franc\\Desktop\\Proyecto_Peliculas\\notebooks\\ETL'

In [59]:
base_dir = (
    os.path.dirname(
        os.path.dirname(
            current_dir
            )))

base_dir

'c:\\Users\\franc\\Desktop\\Proyecto_Peliculas'

In [60]:
file_path = (
    os.path.join(
        base_dir, 
        'data', 
        'ETL', 
        'movies.parquet'
        ))

file_path

'c:\\Users\\franc\\Desktop\\Proyecto_Peliculas\\data\\ETL\\movies.parquet'

In [61]:
movies.to_parquet(file_path)

In [62]:
del movies
gc.collect()

32